In [1]:
import codecs
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from onlykey.client import OnlyKey, Message, KeyFlags, RSATypeFlags
import logging
import time
import binascii
from typing import Optional


In [12]:
def bin2hex(bin_str):
    return binascii.hexlify(bin_str)


def hex2bin(hex_str):
    return binascii.unhexlify(hex_str)

def pack_long(n):
    """this convert 10045587143827198209824131064458461027107542643158086193488942239589004873324146472911535357118684101051965945865943581473431374244810144984918148150975257L
    to "\xbf\xcd\xce\xa0K\x93\x85}\xf0\x18\xb3\xd3L}\x14\xdb\xce0\x00uE,\x05'\xeeW\x1c\xeb\xcf\x8b\x1f\xcc\xc5\xc1\xe2\x17\xb7\xa3\xb6C\x16\xea?\xcchz\xebF1\xb7\xb1\x86\xb8\n}\x82\xebx\xce\x1b\x13\xdf\xdb\x19"
    it seems to be want you wanted? it's 64 bytes.
    """
    h = b'%x' % n
    s = codecs.decode((b'0' * (len(h) % 2) + h), 'hex')
    return s

def get_button(byte):
    ibyte = ord(bytes([byte]))
    if ibyte < 6:
        return 1
    return ibyte % 5 + 1

def get_key_type(key_length):
    if key_length == 1024:
        return RSATypeFlags.RSA1024
    if key_length == 2048:
        return RSATypeFlags.RSA2048
    if key_length == 3072:
        return RSATypeFlags.RSA3072
    if key_length == 4096:
        return RSATypeFlags.RSA4096
    
def get_pub_key_bytes(slot_num, loop_len=20, check_size=64, dev: OnlyKey = None):
    """
    Get the bytes of a public key on a given slot
    :param slot_num: slot number to use
    :param loop_len: How many times to check for bytes
    :param check_size: how many bytes to check at a time.
    :param dev: OnlyKey Device instance.
    :return: bytes
    """
    if dev is None :
        dev = ok
    # Flush the buffer before reading
    for _ in range(0, 10):
        r = dev.read_bytes(timeout_ms=1000)
        if r == '':
            break
    time.sleep(0.25)
    dev.send_message(msg=Message.OKGETPUBKEY, payload=chr(slot_num))
    time.sleep(1.5)
    pb_bytes = []
    for _ in range(loop_len):
        b = dev.read_bytes(check_size, timeout_ms=1000)
        if len(b) == check_size:
            pb_bytes += b
        time.sleep(1)
    return bytes(pb_bytes)

def has_pub_key(slot: int, dev: OnlyKey = None):
    """
    Checks if a slot has a pub key
    :param slot: 
    :param dev: OnlyKey Device Instance
    :return: True if any results
    """
    b = get_pub_key_bytes(slot_num=slot, loop_len=1, check_size=15, dev=dev)
    print(b)
    if b and b != b'There is no RSA':
        return True
    else:
        return False
    


# Putting an RSA key on the Only Key

## Generate a 2048 bit RSA key


In [22]:
priv_key = rsa.generate_private_key(65537, 2048, default_backend())  # type: rsa.RSAPrivateKey
pub_key = priv_key.public_key()  # type: rsa.RSAPublicKey
print('Key Created')
hexPrivKey = bin2hex(
    priv_key.private_bytes(encoding=serialization.Encoding.DER, format=serialization.PrivateFormat.PKCS8,
                           encryption_algorithm=serialization.NoEncryption())
)
hexPubKey = bin2hex(
    pub_key.public_bytes(encoding=serialization.Encoding.DER, format=serialization.PublicFormat.PKCS1)
)


Key Created


## Combine q and p and get n

In [23]:
q_and_p = pack_long(priv_key.private_numbers().q) + pack_long(priv_key.private_numbers().p)

public_n = pack_long(pub_key.public_numbers().n)
print("Public N: ", pub_key.public_numbers().n)

Public N:  71928046108817864407955534850247301435299596711652432121712704457202338646485290823660641310955508908081271431098868675853341672797325071647537056246052088098788865486376423976742319229161830302915637402343023788516570456390192170021453570659521564037614800519530735574308320213424605619432415271829441738978345196388200179217328212857640251748561763588795070290332058379985991236885104143770691096272929382257968756968011134333291774252777673427202248191088608548592496964656775003377314722849393616930783273396565273771488486956533595365756775632923252040437261615421432942145608468734524919461331563261991335573321056169810785305972200510797061435257161234922715838806618681081537926139904221537024073561347070143557438928671909433023958525832040288243143756573416775911261620285971697426602798038369891682260936512074004245628271929884548496792009751926658988401228590438345382122640667388003972675338848481743377746067553724306243722894199348010300934781647214465282473336289610361160

## Send the key to the device
You need to put the device in config mode first.
Hold the 6 key for 5 seconds and release


Get a connection to the OnlyKey

In [38]:
ok = OnlyKey()
time.sleep(2)
print('Connected')


Connected


Get the type of key

(For this demo we are marking it as a Signature, Authentication, and Decrytion key for testing)

In [24]:
key_type = get_key_type(priv_key.key_size) | KeyFlags.SIGNATURE | KeyFlags.AUTHENTICATION | KeyFlags.DECRYPTION
print(key_type)

KeyFlags.SIGNATURE|DECRYPTION|AUTHENTICATION|KEYL4096


Send the keys over

(First check if there is a key)

In [16]:
for i in range(1, 5):
    print("Key {} has key?: ".format(i), has_pub_key(i))
    

b'\xc1\xdbJ\x1e\xd6\xeb\xe97\xafQ\xbb"\xf2\x152'
Key 1 has key?:  True
b'There is no RSA'
Key 2 has key?:  False
b'\xaf=MD\xb0\xc0\xb8\x81\xfb\xa1\xc4\xb7\r-J'
Key 3 has key?:  True
b'There is no RSA'
Key 4 has key?:  False


In [30]:
ok.send_large_message3(msg=Message.OKSETPRIV, slot_id=3, key_type=key_type, payload=list(q_and_p))
time.sleep(10)
print(ok.read_string())


INITIALIZED


## Test Decryption

In [54]:
test_bytes = b"Hello World!"
slot = 3
encrypted = pub_key.encrypt(test_bytes, padding.PKCS1v15())

In [55]:
def compute_challenge_pin(ciphertext: bytes):
    h = hashes.Hash(hashes.SHA256(), default_backend())
    h.update(ciphertext)
    d = h.finalize()
    
    def get_button(byte):
        ibyte = ord(bytes([byte]))
        if ibyte < 6:
            return 1
        return ibyte % 5 + 1
    return get_button(d[0]), get_button(d[15]), get_button(d[31])

print('Challenge Pin: ', compute_challenge_pin(encrypted))


Challenge Pin:  (1, 3, 1)


Send the payload to OnlyKey


In [56]:
ok.send_large_message2(msg=Message.OKDECRYPT, payload=list(encrypted), slot_id=slot)
print('Please enter the 3 digit challenge code on OnlyKey (and press ENTER if necessary)')


Please enter the 3 digit challenge code on OnlyKey (and press ENTER if necessary)


In [57]:
ok_decrypted = ''
while ok_decrypted == '':
    time.sleep(0.5)
    ok_decrypted = ok.read_string()
print(ok_decrypted)


Hello World!
